In [48]:
import pandas as pd
import numpy as np
import os
import math
import re
import random
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
from matplotlib.colors import LinearSegmentedColormap, Normalize
import igraph as ig
from collections import Counter
import scipy.stats
import seaborn as sns
from matplotlib.font_manager import FontProperties  # 导入FontProperties
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
from matplotlib.colors import LinearSegmentedColormap, Normalize
import warnings
warnings.filterwarnings("ignore")

# data import

In [2]:
path1 = r'E:\netData\APS\aps-dataset-citations-2010.zip'
f1 = pd.read_csv(path1)

In [6]:
path2 = r'E:\netData\APS\3 properties\nodepro2010.zip'
f2 = pd.read_csv(path2)

In [3]:
f1 = f1[['citing_doi', 'cited_doi']]

In [7]:
f1['citing_id'] = f1['citing_doi'].map(dict(zip(f2['id'],f2.index)))
f1['cited_id'] =  f1['cited_doi'].map(dict(zip(f2['id'],f2.index)))
f11 = f1[['cited_id','citing_id']].copy()
f11 = f11[f11.columns[::-1]]

In [8]:
g = ig.Graph.DataFrame(f11, directed=True)

In [9]:
g.vcount()

463348

In [10]:
g.ecount()

4745622

In [17]:
g.reciprocity()

0.001745678056000779

# calculate D and CD

In [37]:
def calculate_CD(x):
    if g.vs[x].indegree()== 0:
        return 0,0
    elif g.vs[x].outdegree()== 0:
        return 1,1
    else:
        array = np.array([])
        for successor in g.vs[x].successors():
            array = np.append(array,successor.predecessors())
        array = np.unique(array)
        
        nj = np.intersect1d(array,g.vs[x].predecessors())
        ni = np.setdiff1d(g.vs[x].predecessors(),nj)
        nk = np.setdiff1d(array,g.vs[x].predecessors())

        CD = (len(ni)-len(nj))/(len(ni) + len(nj)+ len(nk))
        
        D = (len(ni))/(len(ni) + len(nj)+ len(nk))
        return [D, CD]

# 2stepD

In [38]:
def calculate_2D(x):
    if g.vs[x].indegree()== 0:
        return 0
    elif g.vs[x].outdegree()== 0:
        return 1
    else:
        array = np.array([])
        for successor in g.vs[x].successors():
            array = np.append(array,successor.predecessors())
        array = np.unique(array)
        
        twogarray = np.array([])
        for prede in g.vs[x].predecessors():
            array = np.append(array,prede.predecessors())
        
        twogarray = np.union1d(g.vs[x].predecessors() , twogarray)
        
        nj = np.intersect1d(array,twogarray)
        ni = np.setdiff1d(twogarray,nj)
        nk = np.setdiff1d(array,twogarray)

        D2 = len(ni)/(len(ni) + len(nj)+ len(nk))
        return D2

# save

In [39]:
M = g.vcount()

In [40]:
Dlst = []
D2lst = []
CDlst = []

In [41]:
for i in range(M):
    DCD = calculate_CD(i)
    Dlst.append(DCD[0])
    CDlst.append(DCD[1])
    D2lst.append(calculate_2D(i))

In [42]:
out_data = [Dlst,D2lst ,CDlst]
name_data = ['D','2stepD','CD']
Ddata = pd.DataFrame(dict(zip(name_data , out_data)))

In [43]:
Ddata.describe()

,D,2stepD,CD
count,463348.000000,463348.000000,463348.000000
mean,0.048016,0.032958,0.023165
std,0.172696,0.162038,0.178672
min,0.000000,0.000000,-1.000000
25%,0.000000,0.000000,-0.013228
50%,0.001054,0.000000,-0.002196
75%,0.012784,0.004800,0.000000
max,1.000000,1.000000,1.000000


In [45]:
f2['CD'] = CDlst
f2['2stepD'] = D2lst
f2['D'] = Dlst

In [49]:
f2['degree'] = g.degree()

In [50]:
f2.to_csv(r'E:\netData\APS\3 properties\nodepro2010.zip',index=False)